# BOOK RECOMMENDER SYSTEM

### A simple content based Book recommender system based on author names and genres.

![](https://www.penguin.co.uk/content/dam/prh/articles/adults/2020/december/John-le-carre-where-to-start.jpg)

__________________________________________________

## Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,sigmoid_kernel

In [ ]:
red=pd.read_csv('best-books-ever_data.csv')

In [ ]:
red

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",Paperback,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...
...,...,...,...,...,...,...,...,...,...,...,...,...
54296,Howard Megdal,"In this fearless and half-crazy story, Howard ...",NaN,Hardcover,9.78161E+12,256 pages,3.37,27,9,Taking the Field: A Fan's Quest to Run the Tea...,Sports|Baseball|Sports and Games|Sports|Nonfic...,https://images.gr-assets.com/books/1312074392l...
54297,Howard Megdal,From the icons of the game to the players who ...,NaN,Hardcover,9.78006E+12,256 pages,3.97,34,5,"The Baseball Talmud: Koufax, Greenberg, and th...",Nonfiction|Sports and Games|Sports,https://images.gr-assets.com/books/1348841629l...
54298,Howard Megdal,NaN,NaN,Kindle Edition,NaN,NaN,3.66,32,3,"Wilpon's Folly - The Story of a Man, His Fortu...",Sports|Baseball|Abandoned,https://images.gr-assets.com/books/1394277097l...
54299,Mimi Baird|Eve Claxton,"Soon to be a major motion picture, from Brad P...",NaN,Hardcover,9.7808E+12,272 pages,3.82,867,187,He Wanted the Moon: The Madness and Medical Ge...,Nonfiction|Autobiography|Memoir|Biography|Psyc...,https://images.gr-assets.com/books/1403192135l...


In [ ]:
red.image_url[0]

'https://images.gr-assets.com/books/1447303603l/2767052.jpg'

In [ ]:
red.describe()

,book_rating,book_rating_count,book_review_count
count,54301.000000,5.430100e+04,54301.00000
mean,4.020027,4.350449e+04,2011.60218
std,0.362100,2.126572e+05,7627.07287
min,0.000000,0.000000e+00,0.00000
25%,3.830000,4.070000e+02,35.00000
50%,4.030000,2.811000e+03,188.00000
75%,4.220000,1.274500e+04,822.00000
max,5.000000,5.588580e+06,160776.00000


In [ ]:
red.describe(include='O')

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_title,genres,image_url
count,54301,52970,5453,52645,41435,51779,54301,51059,53618
unique,27159,51781,2134,147,548,1403,48483,30094,53618
top,Stephen King,"هذه هي طبعة ""دار الفكر - بيروت"" وهي آخر طبعة ع...",First Edition,Paperback,9.78006E+12,320 pages,1984,Fiction,https://images.gr-assets.com/books/1349882466l...
freq,112,38,437,28725,2421,1077,17,234,1


In [ ]:
red.isnull().mean()

book_authors         0.000000
book_desc            0.024512
book_edition         0.899578
book_format          0.030497
book_isbn            0.236939
book_pages           0.046445
book_rating          0.000000
book_rating_count    0.000000
book_review_count    0.000000
book_title           0.000000
genres               0.059704
image_url            0.012578
dtype: float64

In [ ]:
red.drop(red[red['book_desc'].isna()].index,inplace=True)

In [ ]:
red=red.drop(['book_edition','book_isbn','image_url','book_pages'],1)

In [ ]:
red.isna().sum()

book_authors            0
book_desc               0
book_format          1352
book_rating             0
book_rating_count       0
book_review_count       0
book_title              0
genres               2793
dtype: int64

In [ ]:
red=red.dropna()

In [ ]:
pd.unique(red['book_format'])

array(['Hardcover', 'Paperback', 'Mass Market Paperback', 'ebook',
       'Board book', 'Boxed Set', 'Audio CD', 'Capa comum',
       'Kindle Edition', 'Audio Cassette', 'Capa dura', 'Box Set', 'Nook',
       'Library Binding', 'Pasta blanda', 'Slipcased Hardcover',
       'Broschiert', 'Unknown Binding', 'Trade Paperback', 'Board Book',
       'Paperback ', 'MP3 CD', 'Audiobook', 'cloth', 'Brochura',
       'Leather Bound', 'pocket', 'Audio', 'Textbook Binding',
       'Paperback/Kindle', 'paper', 'Perfect Paperback',
       'Digital virtual book', 'Hardcover Slipcased ', 'unknown',
       'Paper back', 'Paperback, Kindle, Ebook, Audio',
       'Paperback and Kindle', 'online serial', 'Klappenbroschur',
       'Audible Audio', 'Hardcover im Schuber', 'Poche', 'paperback',
       'Graphic Novels', 'broché', 'Science Fiction Book Club Omnibus',
       'Newsprint', 'Spiral-bound', 'Mass Market', 'Comic',
       'Hardcover Boxed Set', 'Taschenbuch', 'Mass Market Paperback ',
       'Hardb

In [ ]:
red.book_desc[30]

"A funny, often poignant tale of boy meets girl with a twist: what if one of them couldn't stop slipping in and out of time? Highly original and imaginative, this debut novel raises questions about life, love, and the effects of time on relationships.Audrey Niffenegger’s innovative debut, The Time Traveler’s Wife, is the story of Clare, a beautiful art student, and Henry, an adventuresome librarian, who have known each other since Clare was six and Henry was thirty-six, and were married when Clare was twenty-three and Henry thirty-one. Impossible but true, because Henry is one of the first people diagnosed with Chrono-Displacement Disorder: periodically his genetic clock resets and he finds himself misplaced in time, pulled to moments of emotional gravity in his life, past and future. His disappearances are spontaneous, his experiences unpredictable, alternately harrowing and amusing. The Time Traveler’s Wife depicts the effects of time travel on Henry and Clare’s marriage and their pa

In [ ]:
red.head()

,book_authors,book_desc,book_format,book_rating,book_rating_count,book_review_count,book_title,genres
0,Suzanne Collins,Winning will make you famous. Losing means cer...,Hardcover,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,Paperback,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction
2,Harper Lee,The unforgettable novel of a childhood in a sl...,Paperback,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,Paperback,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance
4,Stephenie Meyer,About three things I was absolutely positive.F...,Paperback,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...


In [ ]:
def to_string(x):
    if x:
        return x.replace('|',' ')
    else:
        return ''
    
red['book_authors']=red['book_authors'].apply(lambda x: to_string(x))
red['genres']=red['genres'].apply(lambda x: to_string(x))

In [ ]:
red.columns

Index(['book_authors', 'book_desc', 'book_format', 'book_rating',
       'book_rating_count', 'book_review_count', 'book_title', 'genres'],
      dtype='object')

In [ ]:
def soup(x):
    return x.book_authors+' '+x.genres

In [ ]:
red['soup']=red.apply(soup, 1)

_________________________________________

# Creating Sample
### Taking a sample out of large dataset

In [ ]:
black = (red.sample(replace = False, # Pick every row once
                         frac = 0.3) # Random state for reproducibility
            .sort_index()
           .reset_index())

In [ ]:
black.drop_duplicates(subset ="book_title", 
                     keep = False, inplace = True) 
  

In [ ]:
black

,index,book_authors,book_desc,book_format,book_rating,book_rating_count,book_review_count,book_title,genres,soup
1,1,J.K. Rowling Mary GrandPré,There is a door at the end of a silent corrido...,Paperback,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy Young Adult Fiction,J.K. Rowling Mary GrandPré Fantasy Young Adult...
6,9,J.R.R. Tolkien,لجزء الثالث من ملحمة جيه أر أر تولكين الرائعة ...,Mass Market Paperback,4.59,99793,1652,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,Fantasy Fiction Classics,J.R.R. Tolkien Fantasy Fiction Classics
10,21,William Shakespeare Barbara A. Mowat Paul Wers...,"In Romeo and Juliet, Shakespeare creates a wor...",Mass Market Paperback,3.74,1818262,16442,Romeo and Juliet,Classics Plays Fiction Romance Academic School...,William Shakespeare Barbara A. Mowat Paul Wers...
13,38,William Goldman,What happens when the most beautiful girl in t...,Paperback,4.26,685278,17221,The Princess Bride,Fantasy Fiction Classics Romance Humor Young A...,William Goldman Fantasy Fiction Classics Roman...
15,45,Rick Riordan,Alternate cover for this ISBN can be found her...,Paperback,4.24,1623502,50794,The Lightning Thief,Fantasy Young Adult Fantasy Mythology Fiction,Rick Riordan Fantasy Young Adult Fantasy Mytho...
...,...,...,...,...,...,...,...,...,...,...
14693,54270,Kazuma Kamachi Kiyotaka Haimura,"In Academy City, magic and science coexist in ...",Paperback,3.99,858,37,"A Certain Magical Index, Vol. 1",Novels Light Novel Fantasy Sequential Art Mang...,Kazuma Kamachi Kiyotaka Haimura Novels Light N...
14694,54271,Kazuma Kamachi Kiyotaka Haimura,「そこ、女の子が捕まってるから」“超能力”が一般科学として認知された学園都市、その「三沢塾」...,文庫,3.88,443,13,とある魔術の禁書目録 2,Novels Light Novel Fantasy Science Fiction Seq...,Kazuma Kamachi Kiyotaka Haimura Novels Light N...
14695,54274,Yoshitoki Oima,"“Tolong jangan lari, Nishimiya!”Ishida Shoya, ...",Paperback,4.37,4300,299,The Shape of Voice 02,Sequential Art Manga Sequential Art Graphic No...,Yoshitoki Oima Sequential Art Manga Sequential...
14696,54287,WAKAKI Tamiki,Target Keima kali ini adalah kapten klub karat...,Paperback,4.45,22,0,The World God Only Knows 03 (The World God Onl...,Sequential Art Manga,WAKAKI Tamiki Sequential Art Manga


In [ ]:
pink=black.sort_values(by='genres',ascending=True)

# TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
tf_mat=tf.fit_transform(black.soup)

In [ ]:
tf_mat.shape

(13444, 12061)

# Computing Sim Scores 

In [ ]:
sig=sigmoid_kernel(tf_mat,tf_mat)

In [ ]:
ind=black.index
titles=black['book_title']
idx=pd.Series(ind,index=titles).drop_duplicates()
idx

book_title
Harry Potter and the Order of the Phoenix                                    1
J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings        6
Romeo and Juliet                                                            10
The Princess Bride                                                          13
The Lightning Thief                                                         15
                                                                         ...  
A Certain Magical Index, Vol. 1                                          14693
とある魔術の禁書目録 2                                                             14694
The Shape of Voice 02                                                    14695
The World God Only Knows 03 (The World God Only Knows, # 3)              14696
Running the Books: The Adventures of an Accidental Prison Librarian      14697
Length: 13444, dtype: int64

In [ ]:
def get_recom(title,sig=sig):
    idcs=idx[title]
    sim_sc=list(enumerate(sig[idcs]))
    sim_sc=sorted(sim_sc,key=lambda x: x[1],reverse=True)
    sim_sc=sim_sc[1:21]
    book_indices=[i[0] for i in sim_sc]
    wss=[]
    for i in book_indices:
        wss.append(black['book_title'].iloc[i])
        
    return wss

## Getting Recommendations

In [ ]:
get_recom(black['book_title'].iloc[8])

['The Marriage Plot',
 'Only Time Will Tell',
 'Dark Prince',
 'The Drawing of the Dark',
 'A Matter of Honor',
 'The Memory Artists',
 'The Fourth Estate',
 'Sons of Fortune',
 'The Eleventh Commandment',
 'Unicorn and Dragon',
 'The Lions of Al-Rassan',
 'The Well of Shades',
 'The Body at the Tower',
 'Brethren',
 'Wild Rover No More: Being the Last Recorded Account of the Life & Times of Jacky Faber',
 "Resolution: A Novel of Captain Cook's Adventures of Discovery to Australia, New Zealand and Hawaii, Through the Eyes of George Forster, the Botanist on Board His Ship",
 'The Wild Hunt',
 'Territory',
 'Dolce sacrificio',
 'Chesapeake']

In [ ]:
black['book_title'].iloc[2]

'Romeo and Juliet'

In [ ]:
black['book_title'][6948]

'Educating Rita'